In [163]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from rest import Rest
import pandas as pd
import numpy as np
from strategies import OptionStrat
pd.set_option('display.max_rows', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [303]:
conn = Rest()
ticker = 'SPY'
call_option_chain = pd.DataFrame(conn.get_option_data(ticker, '2021-05-21', 'call'))
put_option_chain = pd.DataFrame(conn.get_option_data(ticker, '2021-05-21', 'put'))
latest_price = conn.get_latest_price(ticker)

Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data 

In [310]:
#convert to strike prices to float
call_option_chain["strike_price"] = call_option_chain.strike_price.astype(float)
put_option_chain["strike_price"] = put_option_chain.strike_price.astype(float)

#convert delta prices to float
call_option_chain["delta"] = call_option_chain.delta.astype(float)
put_option_chain["delta"] = put_option_chain.delta.astype(float)

delta = 0.16
# # get all rows from call option chain where strike price greater than latest price of underlying and delta greater than 0.16
call_option_chain_filtered = call_option_chain[(call_option_chain.strike_price > latest_price ) & (call_option_chain.delta < delta)]

#get all rows from put option chain below latest price of underlying
put_option_chain_filtered = put_option_chain[(put_option_chain.strike_price < latest_price ) & (put_option_chain.delta > -delta)]

#sort the dataframe based on strike prices
call_option_chain_filtered = call_option_chain_filtered.sort_values(by=['strike_price'], ascending = False)
put_option_chain_filtered = put_option_chain_filtered.sort_values(by=['strike_price'], ascending = False)

In [311]:
call_options_prices = []
for index, row in call_option_chain_filtered.iterrows():
    if float(row['bid_price']) > 0 and float(row['ask_price']) > 0:
        call_options_prices.append({
            'strike_price': row['strike_price'], 
            'bid': float(row['bid_price']), 
            'ask': float(row['ask_price'])
        })

put_options_prices = []
for index, row in put_option_chain_filtered.iterrows():
    if float(row['bid_price']) > 0 and float(row['ask_price']) > 0:
        put_options_prices.append({
            'strike_price': row['strike_price'], 
            'bid': float(row['bid_price']), 
            'ask': float(row['ask_price'])
        })

In [312]:
call_credit_spreads = []
for i in range(len(call_options_prices)-1):
    call_credit_spreads.append({'buy' : call_options_prices[i], 
                                'sell': call_options_prices[i+1]})

put_credit_spreads = []
for i in range(len(put_options_prices)-1):
    put_credit_spreads.append({'sell' : put_options_prices[i], 
                                'buy': put_options_prices[i+1]})

In [313]:
df = pd.DataFrame()

for call_credit_spread in call_credit_spreads:
    for put_credit_spread in put_credit_spreads:
        obj = OptionStrat('Iron Condor', 200, {'start': put_credit_spread['buy']['strike_price'] - 50, 'stop':call_credit_spread['buy']['strike_price']+50,'by':1})
        
        obj.long_call(call_credit_spread['buy']['strike_price'], call_credit_spread['buy']['ask'], 1)        
        obj.short_call(call_credit_spread['sell']['strike_price'], call_credit_spread['sell']['bid'], 1)
        
        obj.short_put(put_credit_spread['sell']['strike_price'], put_credit_spread['sell']['bid'], 1)  
        obj.long_put(put_credit_spread['buy']['strike_price'], put_credit_spread['buy']['ask'], 1)           
        
        # obj.plot(color='black', linewidth=2)  
        # obj.describe()
        max_profit, max_loss, probability_of_success, risk_to_reward_ratio = obj.calculate_metrics()
        df = df.append({
            'long_call_strike':call_credit_spread['buy']['strike_price'], 
            'long_call_strike_ask': call_credit_spread['buy']['ask'],
            'short_call_strike': call_credit_spread['sell']['strike_price'], 
            'short_call_strike_bid': call_credit_spread['sell']['bid'],
            'short_put_strike': put_credit_spread['sell']['strike_price'],
            'short_put_strike_bid':put_credit_spread['sell']['bid'],
            'long_put_strike': put_credit_spread['buy']['strike_price'],
            'long_put_strike_ask': put_credit_spread['buy']['ask'],
            'max_profit' : max_profit, 
            'max_loss': max_loss, 
            'probability_of_success':probability_of_success, 
            'risk_to_reward_ratio':risk_to_reward_ratio,
            'break_even_price': (call_credit_spread['sell']['strike_price'], put_credit_spread['sell']['strike_price'])
            },
            ignore_index = True)

c:\trading_projects\options_strats\strategies.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  risk_to_reward_ratio = round(abs(max_loss / max_profit),3)


In [315]:
pd.set_option('display.max_rows', 100)
df_new = df[(df.probability_of_success > 0.75) & (df.risk_to_reward_ratio != float('inf'))]
df_new = df_new.sort_values(by=['probability_of_success', 'short_call_strike'], ascending = [True, False])
df_new[['short_call_strike','short_call_strike_bid',
        'long_call_strike','long_call_strike_ask',
        'short_put_strike','short_put_strike_bid', 
        'long_put_strike','long_put_strike_ask', 
        'max_loss', 'max_profit', 'risk_to_reward_ratio', 'probability_of_success','break_even_price']]

,short_call_strike,short_call_strike_bid,long_call_strike,long_call_strike_ask,short_put_strike,short_put_strike_bid,long_put_strike,long_put_strike_ask,max_loss,max_profit,risk_to_reward_ratio,probability_of_success,break_even_price
465,132.0,0.24,133.0,0.16,122.0,0.38,121.0,0.33,-0.87,0.13,6.692,0.851,"(132.0, 122.0)"
466,132.0,0.24,133.0,0.16,121.0,0.30,120.0,0.27,-0.89,0.11,8.091,0.876,"(132.0, 121.0)"
467,132.0,0.24,133.0,0.16,120.0,0.25,119.0,0.22,-0.89,0.11,8.091,0.876,"(132.0, 120.0)"
468,132.0,0.24,133.0,0.16,119.0,0.21,118.0,0.19,-0.90,0.10,9.000,0.889,"(132.0, 119.0)"
434,133.0,0.15,134.0,0.11,122.0,0.38,121.0,0.33,-0.91,0.09,10.111,0.901,"(133.0, 122.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,146.0,0.01,148.0,0.02,119.0,0.21,118.0,0.19,-1.99,0.01,199.000,0.995,"(146.0, 119.0)"
36,146.0,0.01,148.0,0.02,117.0,0.15,116.0,0.15,-2.01,-0.01,201.000,0.995,"(146.0, 117.0)"
39,146.0,0.01,148.0,0.02,114.0,0.10,113.0,0.10,-2.01,-0.01,201.000,0.995,"(146.0, 114.0)"
41,146.0,0.01,148.0,0.02,112.0,0.08,111.0,0.08,-2.01,-0.01,201.000,0.995,"(146.0, 112.0)"
